<a href="https://colab.research.google.com/github/anitarmenta/PracticaGit/blob/main/main.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [7]:
# -*- coding: utf-8 -*-
from lxml import etree as etr
from xml.etree import ElementTree
import re
import os
import glob
import json
from collections import defaultdict  


In [8]:
from google.colab import files
xml = []
xml = files.upload()


Saving ABAISSER.xml to ABAISSER.xml
Saving ABATTRE.xml to ABATTRE (1).xml


In [ ]:
for filename in glob.glob( os.path.join("letraA/", '*.xml') ):
  xml = etr.parse(filename)

In [9]:
def get_ortho(xml):    
    ortho = xml.xpath("/entry/form/orth")
    return ortho[0].text.capitalize() # pour mmetre la première lettre de chaque clé en maj 

def get_gram(xml):
    gram = dict()    
    #récup de pos
    pos = xml.find("./gramgrp//*[@type='pos']")
    if pos is not None:
      gram["pos"] = pos.text      
      return gram["pos"]

def get_genre(xml):
    genre=dict()
    gen = xml.find("./gramgrp//*[@type='gen']")
    if gen is not None:
      genre["gen"] = gen.text    
      return genre["gen"]

#Ecriture   

#separate nom and classe

def cle_mot(mot):
    cle = f"=={mot}==\n"
    return cle
  
def get_etym(xml):
    etym = dict()
    etymo = xml.find("./etym/mentioned")
    if etymo:
      etym["etymo"]=etymo.text
      return etym["etymo"]

def get_senses(xml):
    lSenses = xml.findall("./sense")
    return lSenses

def clean_texte(chaine):
    # dictionnaire contenant les mots à remplacer
  substituts = {"<fleur>":" \'\'", "</fleur>":"\'\' ",\
                    "<mark>":" [[Forme orthographique::", "</mark>":"]] ",\
                    "<gloss>":"", "</gloss>":"",\
                    "<quote>":"", "</quote>":"",\
                    "<author>":"[[Référence::", "</author>":"]]",\
                    "<nonit>":"", "</nonit>":"",\
                    "<bibl>":"", "</bibl>":"",\
                    "\n      ":"","\n":""}
    
  tampon = chaine
    
  for mot in enumerate(substituts.keys()):
    if (mot[1] in chaine):
      tampon = tampon.replace(mot[1],substituts[mot[1]])
    else:
      pass        
  return tampon

# dictionnaire des domaines modernes/contemporain
dicodomMod = {'AGR.':'Agronomie','CHIM.':'Chimie',\
              'MAG.':'Magie','ARCHI.':'Architecture',\
              'NAV.':'Navigation','TECH.':'Techniques',\
              'ASTR.':'Astronomie','ASTRL.':'Astrologie',\
              'GEOM.':'Géométrie','MED.':'Médecine',\
              'CHIR.':'Chirurgie','ANAT.':'Anatomie',\
              'DIET.':'Diététique','PHARM.':'Pharmacopée',\
              'MUSIQUE-THEORIE':'Théorie de la musique',\
              'MET.':'Météorologie','PHYS.':'Physique',\
              'OPT.':'Optique','MIN.':'Minéralogie',\
              'BOT.':'Botanique','ZOOL.':'Zoologie',\
              'MATH.':'Mathématique','ARITHM.':'Arithmétique',\
              'ALG.':'Algèbre'}
    # dictionnaire des domaines médiévaux
dicodomMed = {'AGR.':'Agriculture','ALCH.':'Alchimie',\
              'DIVI.':'Divination','MAG.':'Art magique',\
              'Arts_Méca.':'Arts mécaniques','ASTR.':'Astronomie',\
              'GEOM.':'Géométrie','MED.':'Médecine',\
              'Sciences_Nat.':'Sciences de la nature',\
              'MATHS.':'Sciences mathématiques',\
              'MATH.':'Sciences mathématiques'}

def get_domaines(sense):    
    #création d'une liste pour stocker les dictionnaires contenant les noms mediévaux et modernes 
    #raison= le return dans la fonction arrête la loop donc on a qu'une seule variable pour les domaines
    # Maintenant on pourra stocker domaines dans un variable et le reutiliser par la suite example: stocker dans une fichier json 
    domaine=[]  #contient les domaines medievaux, modernes et definitions
    domaineMed=[]#ne contient que les domaines medievaux et leurs définitions (pour l'utiliser dans la page wiki)
    domainesMed=[]
    for sens in sense:        
        domaines = {}
        mod = sens.attrib["mod"]
        if (mod in dicodomMod.keys()):
            mod = dicodomMod[mod]
        med = sens.attrib["med"]
        if (med in dicodomMed.keys()):
            med = dicodomMed[med]       
        domaines["medieval"] = med
        domaines["modern"] = mod
        domaines['Def'] = sens.find("def").text
        #get the first word of the def in variable and replace it with [[motclé::word]]
        domaine.append(domaines)
        domainesMed={}
        domainesMed["medieval"]=med
        domainesMed["Def"]=sens.find("def").text
        c=sens.find("cit/quote")
        if c:
          c=etr.tostring(c, encoding="unicode")
          c=clean_texte(c)
        #//title[@lang='en']
        synonyme=sens.findall("note[@id ='Struct']/xr[@type='syn']/ref")
        synonyme1=""
        for e in synonyme:
          synonyme1+=f'{e.text} '
        if synonyme:
          domainesMed["Def"]+=f"\nSynonyme(s):\n[[synonyme::{synonyme1}]]."
        domainesMed["Def"]+=f"\n Equivalent en sciences modernes: [[Domaine Moderne::{mod}|{mod}]]\n\n'''Citation(s):''' \n {c}"
        biblio=sens.find("cit/bibl")
        if biblio:
          biblio=etr.tostring(biblio, encoding="unicode")
          #use regex to replace []  with ()
          biblio = re.sub( r'\[.*\]', "",biblio)
          biblio=clean_texte(biblio)    
        #adding the date category annotation for Dates [[1230::1234]] in references  
          biblio1 = re.sub(r"([0-9]{4})", r"[[Ciècle::\1]]", biblio)
          regex= re.findall(r"[0-9]{4}", biblio1)
          if regex:
            regex1=get_century(int(regex[0]))
            regex2=f"{str(regex1)}e "
            biblio2=re.sub(r"([0-9]{4})",regex2, biblio1)
            source= f"<ref>{biblio2}</ref>"
            domainesMed["Def"]+=source
        domaineMed.append(domainesMed)        
        print(domaines)
        print(domaineMed)
    return domaine, domaineMed, domainesMed

def show_fact_box(mot):
  fact_box= "__SHOWFACTBOX__"
  return fact_box   
  
def get_century(année):  
  année=int(année)
  return (année + 99) // 100
